In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'uspto-explainable-ai:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-competitions-data%2Fkaggle-v2%2F59575%2F8060720%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240906%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240906T123729Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D884fdd94ff17d0120a3b647511839021d516589dd400696b95eb73334086ab0ad5c86d0c50c8e3ccd1149c51530516b1be044dad3e5dc7af2e7fde917a65e070d4d45ed12361a37153792875840bfbd09fb4d9eb9af16d270856423e5ec0b1fd2adad77b7c7673594d7611bbe98237d0fded34620642a44a8ea3a9756387344377bf2c2c1061f5efe29123d3fdbba57941d2a0c11461ceeee526ce4d61cfbcb9b7727c4ccde0bfa3c8e36614798cc22fc17b9e8ffda601029d3efd7924714db326a643469b556c75d6640153961f42d2163981fd4bfac4558d4e55aa9cf41f0e255d1f4e7ff0baba9c0c2006bcbf54e3b713a29ddce8ae28ff7b329738152436,uspto-explainable-ai-validation-index:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F4892374%2F8246447%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240906%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240906T123729Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3Dce181ff94e9c1efb17c061722a23bbc99a835431781bdc170f921d495fc157f9a41d4879656953feeae08876c42af53d43d467f9cdef6dd33457a1dc7886c68c7aaa27f907e881230c4dad039d20f7b534fca8165ee74227ebb9027f7ac8134ee8ab39a7194f5184b0c06428dfe6af2ce1f7de55a33cc4c330535a263a35af81e5103056b7bd4631852c2bba732f741b5244d677704c1ad5b87b32cc8b3477712c33867533f6a8bfd9c16ee110cc27a8bbbc9c7b7122f283e5e13c9d8800791dffc9353a444f77ed727fde7bdebe89c35d6151afaea323165863f54a6b54409102e59b9b40b383fd02df7b93b68f8874bf8f781e46ffd989a986ff5ab025d6c8,uspto-whoosh-reloaded-2-7-5-patched:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F5007812%2F8413600%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240906%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240906T123729Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3Db7f6c28936900c2a6dbcf8020726741b89d724fa81f27e664c398212d994c5f39f274f138c271783878128835f7cb6add5519dca3bf9356d536a026415216a582594e0158015343cda02175045769ee1991be8cb74c00f1f97d29e9c1ddec80377419dab9fdbf7b4ccf55d4fdeadf92cd167fad5a8b9c1c2c99182626e19c1e45f9873347f0e919aef7db082eefbfd2a7c5bd1f9469765fee7dd171406b91af8876936c969502b2d7c24d2b798059dc4caf47ead1d25ae90c35591ada98ed1f7a55aca7a071b89f70f9bece500b00bca53399039b2b2ed1b70162554a5fff240bba33a57720e9f240db810f8f4f7c61570a851b5a17ca45721bb2a718164cbbd,whoosh-wheel-2-7-4:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F4517815%2F8479599%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240906%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240906T123729Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D49ebf877af2f0752877ca2bb322acee31ac5ed729cc36716c600ecb214a306f0c4b8f17fcd204ba650f0668de3b778e1925c693f8426fbd244b647d7e433a6e0684c5c1eae4fa7df2c068498b18c570ba55e0ba76a98c042b8a1dd43948270cc542617babff35b185842ce5a7e4e573689d0385465e58d6a8c4aa7e08b40ada9c37afa0176d7e396ef007071697d2b81cd375510841da0052989a58ccf44b7645e60cd05c889cefa513dfb4297a1ab1d100729cce6d4a4454d89fdfc16f16169291bfb775949b6d9dea23a1c83621c0828d20135a9230702f2b243947ba45b94b069cfc91896ea23a9d0f0dd165bad61cead7b4e6a262c845118339dec2f3ff2,uspto-ti-cpc-tfidf:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F5109610%2F8553271%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240906%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240906T123729Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D6dfdec82e0a093aab2954ec865bbc18d638ac95bc7b444ebef497946a9ba6ed8a1f88161fa0e7b9558fd4cecafa0946593d7c0ae49a624236ef0f3ad603b8fe5771455a45dee5b94a60fd88cf6fba4b4047fcd2ba8b3f429d466b143d946405efa299a8202ff28d75683007de8b78eafedb0733897a9881bc58391b40acf169a7266860f3fc2cfdbcf00efd3f47abf02c6650d60bcd1a66c3fcb3b2c1ad56fa65de8ee3afa27d08232cbc0fddc006c04d7c907d79b208afd405c019b8e36a9be99fc6584616f380a981cefdbc2749b1b6c9929f9dc1be7807169df4ae9cea763481a43cbaf63e435a07a24506ec2e06a54c502b87d8b54d920018b3754f22792'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
!pip install /kaggle/input/whoosh-wheel-2-7-4/Whoosh-2.7.4-py2.py3-none-any.whl

Processing /kaggle/input/whoosh-wheel-2-7-4/Whoosh-2.7.4-py2.py3-none-any.whl


Based on https://www.kaggle.com/code/tubotubo/uspto-simulated-annealing-baseline

# EDA

In [ ]:
# from pathlib import Path
# import polars as pl
# from tqdm import tqdm
# import whoosh_utils
# comp_data_dir = Path("/kaggle/input/uspto-explainable-ai")

In [ ]:
# # Metadata.parquet
# meta = pl.scan_parquet(comp_data_dir / "patent_metadata.parquet")
# df = meta.head(100).collect()

# print(df)

In [ ]:
# # Test.csv
# test_nn = pl.scan_csv(comp_data_dir / "test.csv")
# test_nn = test_nn.head(10).collect()
# print('test_nn', type(test_nn))
# print(test_nn)


In [ ]:
# # Nearest Neighbor.csv
# test_nn = pl.scan_csv(comp_data_dir / "nearest_neighbors.csv")
# test_nn = test_nn.head(10).collect()
# print('test_nn', type(test_nn))
# print(test_nn)

# Create Test Index

In [ ]:
# %%python
from pathlib import Path
import polars as pl
from tqdm import tqdm
import whoosh_utils

comp_data_dir = Path("/kaggle/input/uspto-explainable-ai")

# Read patent since 1975
meta = pl.scan_parquet(comp_data_dir / "patent_metadata.parquet")

meta = (
    meta.with_columns(
        pl.col("publication_date").dt.year().alias("year"),
        pl.col("publication_date").dt.month().alias("month"),
    )
    .filter(pl.col("publication_date") >= pl.date(1900, 1, 1))
    .rename({"cpc_codes": "cpc"})
    .collect()
)
# print('meat head', meta.head())

test_nn = pl.scan_csv(comp_data_dir / "test.csv")
# test_nn = pl.read_csv(comp_data_dir / "nearest_neighbors.csv").sample(2500).lazy()


# Filtering only the patent meta-information that appears in the test
all_pub = test_nn.melt().collect().get_column("value").unique()
meta = meta.filter(pl.col("publication_number").is_in(all_pub))
print('filtered meta: ', meta.shape, meta.head(5))

# Join meta information
patents = []
n_unique = meta.select(["year", "month"]).n_unique()
print("Unique months: ", n_unique)
for (year, month), _ in tqdm(meta.group_by(["year", "month"]), total=n_unique):
    patent_path = comp_data_dir / f"patent_data/{year}_{month}.parquet"
    patent = pl.scan_parquet(patent_path).select(pl.exclude(["claims", "description"]))
    patents.append(patent)
patent: pl.LazyFrame = pl.concat(patents)

patent = patent.with_columns(
    pl.lit("").alias("claims"),
    pl.lit("").alias("description"),
)
df = patent.head(5).collect()
print('patent: ', df)

meta_with_text = (
    meta.lazy().join(patent, on="publication_number", how="left").collect(streaming=True)
)
meta_with_text.write_parquet("meta_with_text.parquet")
print('meta_with_text', meta_with_text.head(5))

# create index
documents = meta_with_text.to_dicts()
Path("test_index").mkdir(parents=True, exist_ok=True)
whoosh_utils.create_index("test_index", documents)

Processing /kaggle/input/whoosh-wheel-2-7-4/Whoosh-2.7.4-py2.py3-none-any.whl
Whoosh is already installed with the same version as the provided wheel. Use --force-reinstall to force an installation of the wheel.
filtered meta:  (498, 7) shape: (5, 7)
┌─────────────────┬─────────────────┬────────────────┬─────────────┬────────────────┬──────┬───────┐
│ publication_num ┆ publication_dat ┆ filing_date    ┆ family_id   ┆ cpc            ┆ year ┆ month │
│ ber             ┆ e               ┆ ---            ┆ ---         ┆ ---            ┆ ---  ┆ ---   │
│ ---             ┆ ---             ┆ datetime[μs]   ┆ f64         ┆ list[str]      ┆ i32  ┆ i8    │
│ str             ┆ datetime[μs]    ┆                ┆             ┆                ┆      ┆       │
╞═════════════════╪═════════════════╪════════════════╪═════════════╪════════════════╪══════╪═══════╡
│ US-10015385-B2  ┆ 2018-07-03      ┆ 2017-05-18     ┆ 5.246991e7  ┆ ["H04N21/4223" ┆ 2018 ┆ 7     │
│                 ┆ 00:00:00        ┆ 00:0

100%|██████████| 355/355 [00:04<00:00, 73.54it/s]


patent:  shape: (5, 5)
┌────────────────────┬───────────────────────────────────┬──────────┬────────┬─────────────┐
│ publication_number ┆ title                             ┆ abstract ┆ claims ┆ description │
│ ---                ┆ ---                               ┆ ---      ┆ ---    ┆ ---         │
│ str                ┆ str                               ┆ str      ┆ str    ┆ str         │
╞════════════════════╪═══════════════════════════════════╪══════════╪════════╪═════════════╡
│ US-2411842-A       ┆ Coating applying device           ┆          ┆        ┆             │
│ US-2411843-A       ┆ Compensating means for electrica… ┆          ┆        ┆             │
│ US-2411844-A       ┆ Poppet valve and guide assembly   ┆          ┆        ┆             │
│ US-2411845-A       ┆ Cooling system for aircraft engi… ┆          ┆        ┆             │
│ US-2411846-A       ┆ Electrical attachment plug        ┆          ┆        ┆             │
└────────────────────┴─────────────────────────

# Annealing

Optimize the query using the annealing method.

For simplicity, we will create a query using only “OR”.
This means that we will create a query “word1 OR word2 OR word3 OR ...”.

We will use an annealing method to determine which words to use.
The specific steps are as follows

1. select topk words with high TFIDF values as candidates
2. select the word with the maximum AP@50 using the annealing method
3. Combine the selected words with “OR” to form a query

In [ ]:
# https://github.com/perrygeo/simanneal/blob/master/simanneal/anneal.py
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals
import abc
import copy
import datetime
import math
import pickle
import random
import signal
import sys
import time


def round_figures(x, n):
    """Returns x rounded to n significant figures."""
    return round(x, int(n - math.ceil(math.log10(abs(x)))))


def time_string(seconds):
    """Returns time in seconds as a string formatted HHHH:MM:SS."""
    s = int(round(seconds))  # round to nearest second
    h, s = divmod(s, 3600)  # get hours and remainder
    m, s = divmod(s, 60)  # split remainder into minutes and seconds
    return "%4i:%02i:%02i" % (h, m, s)


class Annealer(object):
    """Performs simulated annealing by calling functions to calculate
    energy and make moves on a state.  The temperature schedule for
    annealing may be provided manually or estimated automatically.
    """

    __metaclass__ = abc.ABCMeta

    # defaults
    Tmax = 25000.0
    Tmin = 2.5
    steps = 50000
    max_time = 8  # seconds
    updates = 100
    copy_strategy = "deepcopy"
    user_exit = False
    save_state_on_exit = False

    # placeholders
    best_state = None
    best_energy = None
    start = None

    def __init__(self, initial_state=None, load_state=None):
        if initial_state is not None:
            self.state = self.copy_state(initial_state)
        elif load_state:
            self.load_state(load_state)
        else:
            raise ValueError(
                "No valid values supplied for neither \
            initial_state nor load_state"
            )

        signal.signal(signal.SIGINT, self.set_user_exit)

    def save_state(self, fname=None):
        """Saves state to pickle"""
        if not fname:
            date = datetime.datetime.now().strftime("%Y-%m-%dT%Hh%Mm%Ss")
            fname = date + "_energy_" + str(self.energy()) + ".state"
        with open(fname, "wb") as fh:
            pickle.dump(self.state, fh)

    def load_state(self, fname=None):
        """Loads state from pickle"""
        with open(fname, "rb") as fh:
            self.state = pickle.load(fh)

    @abc.abstractmethod
    def move(self):
        """Create a state change"""
        pass

    @abc.abstractmethod
    def energy(self):
        """Calculate state's energy"""
        pass

    def set_user_exit(self, signum, frame):
        """Raises the user_exit flag, further iterations are stopped"""
        self.user_exit = True

    def set_schedule(self, schedule):
        """Takes the output from `auto` and sets the attributes"""
        self.Tmax = schedule["tmax"]
        self.Tmin = schedule["tmin"]
        self.steps = int(schedule["steps"])
        self.updates = int(schedule["updates"])

    def copy_state(self, state):
        """Returns an exact copy of the provided state
        Implemented according to self.copy_strategy, one of

        * deepcopy: use copy.deepcopy (slow but reliable)
        * slice: use list slices (faster but only works if state is list-like)
        * method: use the state's copy() method
        """
        if self.copy_strategy == "deepcopy":
            return copy.deepcopy(state)
        elif self.copy_strategy == "slice":
            return state[:]
        elif self.copy_strategy == "method":
            return state.copy()
        else:
            raise RuntimeError(
                "No implementation found for " + 'the self.copy_strategy "%s"' % self.copy_strategy
            )

    def update(self, *args, **kwargs):
        """Wrapper for internal update.

        If you override the self.update method,
        you can chose to call the self.default_update method
        from your own Annealer.
        """
        self.default_update(*args, **kwargs)

    def default_update(self, step, T, E, acceptance, improvement):
        """Default update, outputs to stderr.

        Prints the current temperature, energy, acceptance rate,
        improvement rate, elapsed time, and remaining time.

        The acceptance rate indicates the percentage of moves since the last
        update that were accepted by the Metropolis algorithm.  It includes
        moves that decreased the energy, moves that left the energy
        unchanged, and moves that increased the energy yet were reached by
        thermal excitation.

        The improvement rate indicates the percentage of moves since the
        last update that strictly decreased the energy.  At high
        temperatures it will include both moves that improved the overall
        state and moves that simply undid previously accepted moves that
        increased the energy by thermal excititation.  At low temperatures
        it will tend toward zero as the moves that can decrease the energy
        are exhausted and moves that would increase the energy are no longer
        thermally accessible."""

        elapsed = time.time() - self.start
        if step == 0:
            print(
                "\n Temperature        Energy    Accept   Improve     Elapsed   Remaining",
                file=sys.stderr,
            )
            print(
                "\r{Temp:12.5f}  {Energy:12.2f}                      {Elapsed:s}            ".format(
                    Temp=T, Energy=E, Elapsed=time_string(elapsed)
                ),
                file=sys.stderr,
                end="",
            )
            sys.stderr.flush()
        else:
            remain = (self.steps - step) * (elapsed / step)
            print(
                "\r{Temp:12.5f}  {Energy:12.2f}   {Accept:7.2%}   {Improve:7.2%}  {Elapsed:s}  {Remaining:s}".format(
                    Temp=T,
                    Energy=E,
                    Accept=acceptance,
                    Improve=improvement,
                    Elapsed=time_string(elapsed),
                    Remaining=time_string(remain),
                ),
                file=sys.stderr,
                end="",
            )
            sys.stderr.flush()

    def anneal(self):
        """Minimizes the energy of a system by simulated annealing.

        Parameters
        state : an initial arrangement of the system

        Returns
        (state, energy): the best state and energy found.
        """
        step = 0
        self.start = time.time()

        # Precompute factor for exponential cooling from Tmax to Tmin
        if self.Tmin <= 0.0:
            raise Exception(
                'Exponential cooling requires a minimum "\
                "temperature greater than zero.'
            )
        Tfactor = -math.log(self.Tmax / self.Tmin)

        # Note initial state
        T = self.Tmax
        E = self.energy()
        prevState = self.copy_state(self.state)
        prevEnergy = E
        self.best_state = self.copy_state(self.state)
        self.best_energy = E
        trials = accepts = improves = 0
        if self.updates > 0:
            updateWavelength = self.steps / self.updates
            self.update(step, T, E, None, None)

        # Attempt moves to new states
        while (
            (step < self.steps)
            and (not self.user_exit)
            and ((time.time() - self.start) <= self.max_time)
        ):
            step += 1
            T = self.Tmax * math.exp(Tfactor * step / self.steps)
            dE = self.move()
            if dE is None:
                E = self.energy()
                dE = E - prevEnergy
            else:
                E += dE
            trials += 1
            if dE > 0.0 and math.exp(-dE / T) < random.random():
                # Restore previous state
                self.state = self.copy_state(prevState)
                E = prevEnergy
            else:
                # Accept new state and compare to best state
                accepts += 1
                if dE < 0.0:
                    improves += 1
                prevState = self.copy_state(self.state)
                prevEnergy = E
                if E < self.best_energy:
                    self.best_state = self.copy_state(self.state)
                    self.best_energy = E
            if self.updates > 1:
                if (step // updateWavelength) > ((step - 1) // updateWavelength):
                    self.update(step, T, E, accepts / trials, improves / trials)
                    trials = accepts = improves = 0

        self.state = self.copy_state(self.best_state)
        if self.save_state_on_exit:
            self.save_state()

        # Return best state and energy
        return self.best_state, self.best_energy

In [ ]:
import pickle
from dataclasses import dataclass
from pathlib import Path

import numpy as np
from numpy.typing import NDArray
import polars as pl
from tqdm import tqdm
from typing import Any
import whoosh_utils

In [ ]:
def select_top_k_columns(X: Any, k: int) -> tuple[Any, NDArray]:
    row_sums = X.sum(axis=0)
    top_k_indices = np.argsort(-row_sums.A1)[:k]
    X_top = X[:, top_k_indices]
    return X_top, top_k_indices


# https://www.kaggle.com/competitions/uspto-explainable-ai/discussion/499981#2791642
def ap50(preds: list[str], labels: list[str]) -> float:
    precisions = list()
    n_found = 0
    for e, i in enumerate(preds):
        if i in labels:
            n_found += 1
        precisions.append(
            n_found / (e + 1)
        )  # this is the line that is probably incorrect for competition
    return sum(precisions) / 50


@dataclass
class Word:
    category: str
    content: str

    def to_str(self):
        return f"{self.category}:{self.content}"


@dataclass
class State:
    words: list[Word]

    def __post_init__(self):
        self.use = np.random.binomial(1, 0.5, len(self.words))

    def to_query(self):
        words = [word.to_str() for word, use in zip(self.words, self.use) if use]

        return " OR ".join(words)

    def move_1(self):
        """Change whether word is used or not"""
        idx = np.random.choice(len(self.words))
        self.use[idx] = 1 - self.use[idx]
        return self


class USPTOProblem(Annealer):
    def __init__(
        self,
        qp: Any,
        searcher: Any,
        target: list[str],
        init_state: State,
        tmax: int = 30,
        tmin: int = 10,
        steps: int = 100,
        max_time: int = 8,
        copy_strategy: str = "deepcopy",
    ):
        super(USPTOProblem, self).__init__(init_state)
        self.qp = qp
        self.searcher = searcher
        self.target = target
        self.Tmax = tmax
        self.Tmin = tmin
        self.steps = steps
        self.max_time = max_time
        self.copy_strategy = copy_strategy

    def move(self):
        self.state.move_1()

    def energy(self):
        query = self.state.to_query()
        cand = whoosh_utils.execute_query(query, self.qp, self.searcher)
        ap50_score = ap50(cand, self.target)

        return -ap50_score

In [ ]:
comp_data_dir = Path("/kaggle/input/uspto-explainable-ai")
tfidf_dir = Path("/kaggle/input/uspto-ti-cpc-tfidf")

# nearest neighbors and meta
test = pl.read_csv(comp_data_dir / "test.csv")
test_meta = pl.read_parquet("meta_with_text.parquet")

# test index
test_idx = whoosh_utils.load_index("./test_index")
searcher = whoosh_utils.get_searcher(test_idx)
qp = whoosh_utils.get_query_parser()

# for tfidf pickle
def identity(x: Any) -> Any:
    return x

# 2 tfidf pickles
with open(tfidf_dir / "tfidf.pkl", "rb") as f:
    ti_tfidf = pickle.load(f)
with open(tfidf_dir / "cpc_cv_tfidf.pkl", "rb") as f:
    cpc_cv_tfidf = pickle.load(f)

print('ti_tfidf', type(ti_tfidf), ti_tfidf)

/opt/conda/lib/python3.10/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator TfidfTransformer from version 1.4.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator TfidfVectorizer from version 1.4.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


ti_tfidf <class 'sklearn.feature_extraction.text.TfidfVectorizer'> TfidfVectorizer(stop_words='english')


/opt/conda/lib/python3.10/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator CountVectorizer from version 1.4.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator Pipeline from version 1.4.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [ ]:
# print(test)
# print(test[0])
# print(test[i].to_numpy())
# print(test[i].to_numpy().flatten())
# print(test[i].to_numpy().flatten()[1:].tolist())

In [ ]:
scores = []
results = []

for i in tqdm(range(len(test))):
    target = test[i].to_numpy().flatten()[1:].tolist()
    meta_i = test_meta.filter(pl.col("publication_number").is_in(target))

    if len(meta_i) == 0:
        # append dummy
        results.append({"publication_number": test[i, "publication_number"], "query": "ti:device"})
        print("\t Append Dummy", i)
        continue

    # TF-IDF matrix
    ti_mat = ti_tfidf.transform(meta_i.get_column("title").fill_null(""))
    cpc_mat = cpc_cv_tfidf.transform(meta_i.get_column("cpc"))

    # Important topk words
    X_ti, idx = select_top_k_columns(ti_mat, k=5)
    X_cpc, cpc_idx = select_top_k_columns(cpc_mat, k=20)

    # Initialize State with topk words
    # The initial query is "word1 OR word2 OR word3 OR ...""
    topk_words = ti_tfidf.get_feature_names_out()[idx].tolist()
    topk_cpc = cpc_cv_tfidf.get_feature_names_out()[cpc_idx].tolist()

    topk_words = [Word(category="ti", content=x) for x in topk_words]
    topk_cpc = [Word(category="cpc", content=x) for x in topk_cpc]
    words = topk_words + topk_cpc

    state = State(words=words)

    # Determine which words to use using the annealing method
    # 2500 * 5 / 60 / 60 = 3.47 [hour]
    problem = USPTOProblem(qp, searcher, target, state, steps=1000, max_time=10)
    solution, score = problem.anneal()
    print(f"\t Problem Number {i} Score:", -score)
    scores.append(-score)

    # save publication number and query
    results.append(
        {"publication_number": test[i, "publication_number"], "query": solution.to_query()}
    )


print("Average Score:", sum(scores) / len(scores))

  0%|          | 0/10 [00:00<?, ?it/s]

  (0, 365909)	0.4758250797700062
  (0, 356828)	0.5370024723680038
  (0, 235747)	0.4408742694022959
  (0, 197705)	0.3579180315038494
  (0, 195798)	0.40341467429423034
  (1, 356828)	0.5184588610677426
  (1, 322077)	0.6213857044193623
  (1, 241987)	0.14572194828004092
  (1, 235747)	0.42565012890992177
  (1, 199726)	0.3329708177128307
  (1, 34045)	0.17831918523725096
  (2, 339513)	0.35126712969109875
  (2, 329043)	0.34602408784382077
  (2, 322050)	0.40687612154754815
  (2, 248913)	0.3239205842395804
  (2, 241987)	0.10885526543328196
  (2, 118036)	0.30319648475187333
  (2, 92290)	0.35347393296407703
  (2, 90339)	0.25353271060049654
  (2, 80128)	0.3000205898278627
  (2, 56538)	0.32154847678400306
  (3, 403470)	0.3442178603640457
  (3, 388355)	0.3180345978670089
  (3, 384323)	0.2575102562797841
  (3, 273578)	0.4006501460931102
  :	:
  (45, 82725)	0.5201986360657651
  (46, 356828)	0.48837733774837105
  (46, 315964)	0.4156666785666678
  (46, 287930)	0.4421272474992256
  (46, 241987)	0.137267009


 Temperature        Energy    Accept   Improve     Elapsed   Remaining
 10%|█         | 1/10 [00:07<01:09,  7.71s/it]     0:00:07     0:00:00

	 Problem Number 0 Score: 0.9783148555444379
  (0, 346508)	0.6190084450312069
  (0, 249447)	0.523907118386899
  (0, 91700)	0.5851067221315981
  (1, 407060)	0.3186528620214845
  (1, 327335)	0.47899645490538445
  (1, 255823)	0.4475515217952468
  (1, 201862)	0.4085894818122054
  (1, 141671)	0.32490590495114974
  (1, 111859)	0.44295730418704843
  (2, 234197)	0.5408984306273293
  (2, 195933)	0.4833465320805872
  (2, 166615)	0.4383976741571292
  (2, 166253)	0.2660033514909016
  (2, 131975)	0.25809827313547684
  (2, 62389)	0.3797894092313192
  (3, 344339)	0.6046722951096272
  (3, 92804)	0.3947897813605147
  (3, 92107)	0.6175670586787011
  (3, 47792)	0.3116462290725023
  (4, 308794)	0.5922064197215192
  (4, 234072)	0.5096476395269741
  (4, 91705)	0.6241400804027917
  (5, 414516)	0.35250646337078173
  (5, 402474)	0.35681757227488303
  (5, 241987)	0.15682295987956998
  :	:
  (44, 128349)	0.6839463507440404
  (45, 375259)	0.5761778905776571
  (45, 125157)	0.4991996794526787
  (45, 105220)	0.34635


 Temperature        Energy    Accept   Improve     Elapsed   Remaining
 20%|██        | 2/10 [00:09<00:32,  4.10s/it]     0:00:01     0:00:07

...

	 Problem Number 7 Score: 0.86
  (0, 407242)	0.4709874829519587
  (0, 405263)	0.4407277991614189
  (0, 387693)	0.5945088312039667
  (0, 158343)	0.48009274892400083
  (1, 345804)	0.6217017209700171
  (1, 79468)	0.7832540904093122
  (2, 377334)	0.35490356112989035
  (2, 346182)	0.5274790606262647
  (2, 342918)	0.4788250026672592
  (2, 341070)	0.37968699662378963
  (2, 231765)	0.23862687370049407
  (2, 226061)	0.4067319995544851
  (3, 384787)	0.34705489985458465
  (3, 341070)	0.278199282535688
  (3, 327083)	0.5187869153310366
  (3, 325656)	0.3177898013346576
  (3, 313646)	0.2754562159999835
  (3, 253692)	0.2768286904193878
  (3, 223627)	0.2504913972176863
  (3, 180592)	0.3135122053007339
  (3, 177354)	0.2635171832516647
  (3, 90444)	0.22145075014256968
  (4, 403474)	0.6157128634043308
  (4, 96700)	0.6303905008904321
  (4, 72679)	0.47276366847035733
  :	:
  (45, 241987)	0.1398396092855485
  (45, 112935)	0.17066899654935375
  (45, 92624)	0.5258139912969388
  (46, 350023)	0.6661868589029503



 Temperature        Energy    Accept   Improve     Elapsed   Remaining
 90%|█████████ | 9/10 [00:58<00:06,  6.74s/it]     0:00:06     0:00:00

	 Problem Number 8 Score: 0.9123268041976693
  (0, 166267)	0.5275036128343075
  (0, 164584)	0.43900414992511744
  (0, 13914)	0.727334376195212
  (1, 166267)	0.5871033938954126
  (1, 13914)	0.8095119547458753
  (2, 301545)	1.0
  (3, 151066)	0.5642244027308982
  (3, 80134)	0.7841414548863884
  (3, 34045)	0.25840472536627335
  (4, 166267)	0.5275036128343075
  (4, 164584)	0.43900414992511744
  (4, 13914)	0.727334376195212
  (5, 166251)	0.49017456724416003
  (5, 164584)	0.4313534968658064
  (5, 34045)	0.25084995322468384
  (5, 13914)	0.7146589083862421
  (6, 166267)	0.5275036128343075
  (6, 164584)	0.43900414992511744
  (6, 13914)	0.727334376195212
  (7, 166267)	0.5275036128343075
  (7, 164584)	0.43900414992511744
  (7, 13914)	0.727334376195212
  (8, 220651)	0.5754041692268286
  (8, 13914)	0.817869208392383
  (9, 105518)	1.0
  :	:
  (31, 164584)	0.43900414992511744
  (31, 13914)	0.727334376195212
  (32, 166267)	0.5275036128343075
  (32, 164584)	0.43900414992511744
  (32, 13914)	0.7273343761


 Temperature        Energy    Accept   Improve     Elapsed   Remaining
100%|██████████| 10/10 [01:05<00:00,  6.59s/it]    0:00:07     0:00:00

	 Problem Number 9 Score: 0.9597500291885265
Average Score: 0.9568593588182974


In [ ]:
# Remove unwanted files and directories that may cause submission errors
!rm -rf /kaggle/working/*

In [ ]:
submission = pl.DataFrame(results)
submission.write_csv("submission.csv")

submission

publication_number,query
str,str
"""US-2017082634-…","""ti:mass OR ti:…"
"""US-2017180470-…","""ti:method OR t…"
"""US-2018029544-…","""ti:module OR t…"
"""US-2022408153-…","""ti:content OR …"
"""US-2268569-A""","""ti:loading OR …"
"""US-3371854-A""","""ti:pump OR ti:…"
"""US-3589189-A""","""ti:meter OR ti…"
"""US-3881203-A""","""ti:clip OR ti:…"
"""US-4845770-A""","""ti:reader OR t…"


In [ ]:
# for i in range(len(submission['query'])):
#     num_tokens = whoosh_utils.count_query_tokens(submission['query'][i])
#     print(num_tokens)